In [2]:
import re
import math
import json

import numpy as np
import pandas as pd

## Готовим данные

In [3]:
fields = [
    l for l in open('data_description.txt', 'r').read().split('\n')
    if len(l.strip()) > 0
]

In [4]:
columns = [f.strip().split(': ') for f in fields if not f.startswith(' ')]
col_names = {c[0]: f"{c[0]}: {c[1]}" for c in columns}
col_names['SalePrice'] = 'SalePrice'

In [5]:
last_key = None
fv = {}

for f in fields:
    if not f.startswith(' '):
        last_key = f.split(': ')[0]
        continue
    value_pair = f.strip().split('\t')
    fv[(last_key, value_pair[0])] = value_pair[1]

In [6]:
s = pd.read_csv('train.csv')
s.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
row_dict = s.iloc[0]

facts = []
for k in row_dict.keys():
    if k not in col_names:
        continue
    if str(row_dict[k]) == 'nan':
        continue
    facts.append(col_names[k])
    if (k, str(row_dict[k])) in fv:
        facts.append(f'- {fv[(k, str(row_dict[k]))]}')
    else:
        facts.append(f'- {row_dict[k]}')

request = '\n'.join(facts).replace('\n- ', ': ')
request = re.sub('\n[^:]+: (?:Identifies )?', '\n', request)
request = re.sub('\n(\d+)', '\nPrice: $\\1', request)

In [8]:
print(request)

MSSubClass: Identifies the type of dwelling involved in the sale.: 2-STORY 1946 & NEWER
the general zoning classification of the sale.: Residential Low Density
Linear feet of street connected to property: 65.0
Lot size in square feet: 8450
Type of road access to property: Paved
General shape of property: Regular
Flatness of the property: Near Flat/Level
Type of utilities available: All public Utilities (E,G,W,& S)
Lot configuration: Inside lot
Slope of property: Gentle slope
Physical locations within Ames city limits: College Creek
Proximity to various conditions: Normal
Proximity to various conditions (if more than one is present): Normal
Type of dwelling: Single-family Detached
Style of dwelling: Two story
Rates the overall material and finish of the house: Good
Rates the overall condition of the house: Average
Original construction date: 2003
Remodel date (same as construction date if no remodeling or additions): 2003
Type of roof: Gable
Roof material: Standard (Composite) Shingle
E

## Делаем запросы в АПИ

In [19]:
import requests

In [25]:
result = requests.post(
    url='https://llm.api.cloud.yandex.net/llm/v1alpha/instruct',
    headers={'Authorization': f'Bearer {iamtoken}', 'x-folder-id': dir_id},
    json={
      "model": "general",
      "instruction_text": "Ниже перечислены характеристики недвижимости.",
      "request_text": f"{request}\n\nНа основании этой информации о недвижимости ниже напиши продающий текст на русском. Текст не должен содержать лжи и полностью соответствовать данным из характеристик.",
      "generation_options": {
        "max_tokens": 1500,  
        "temperature": 0.5
      }
    }
)

In [26]:
print(result.json()['result']['alternatives'][0]['text'])

Заголовок: Продаётся 2-х этажный дом в Амесе, 2003 года постройки

Текст: Дом расположен в районе Колледж Крик, в городе Амес. Общая площадь дома составляет 196 квадратных метров. На первом этаже находятся 2 спальни, кухня и ванная комната. На втором этаже - 2 спальни и ванная комната. В доме есть все необходимые удобства, включая газовое отопление, кондиционер и электричество. Гараж вмещает 2 автомобиля.

Участок земли площадью 8450 квадратных метров имеет ровную поверхность и находится на тихой улице с асфальтовым покрытием. Дом имеет хорошую отделку и находится в хорошем состоянии. Цена продажи составляет $208 500. Это отличное предложение для тех, кто ищет уютный дом в тихом районе.


## Fewshot

In [27]:
import pickle

In [28]:
train_dataset = pickle.load(open('fewshot.pkl', 'rb'))

In [29]:
print("\n------\n\n".join(v['request'] + "\n-----\n" + v['response'] for v in train_dataset))

Ниже перечислены характеристики недвижимости.
MSSubClass: Identifies the type of dwelling involved in the sale.: 1-1/2 STORY FINISHED ALL AGES
the general zoning classification of the sale.: Residential Low Density
Linear feet of street connected to property: 85.0
Lot size in square feet: 14115
Type of road access to property: Paved
General shape of property: Slightly irregular
Flatness of the property: Near Flat/Level
Type of utilities available: All public Utilities (E,G,W,& S)
Lot configuration: Inside lot
Slope of property: Gentle slope
Physical locations within Ames city limits: Mitchell
Proximity to various conditions: Normal
Proximity to various conditions (if more than one is present): Normal
Type of dwelling: Single-family Detached
Style of dwelling: One and one-half story: 2nd level finished
Rates the overall material and finish of the house: Average
Rates the overall condition of the house: Average
Original construction date: 1993
Remodel date (same as construction date if n

## Finetune

In [30]:
df = pd.read_json('squad.json', lines=True)

In [31]:
df

,ans_starts,ans_texts,context,id,question,title
0,[427],[mobile phones],The difference in the above factors for the ca...,57306bf68ab72b1400f9c4dc,What is one use that would require an antenna ...,Antenna_(radio)
1,[1020],[300],The coronation of Charlemagne as emperor on Ch...,56fb69e38ddada1400cd63f7,About how many counts existed in the Carolingi...,Middle_Ages
2,[339],[fossil pollen deposits in sediments],Plant responses to climate and other environme...,5726ccf9f1498d1400e8eb95,How can climate changes be determined from soil?,Botany
3,[347],[KMSB-TV 11],The Tucson metro area is served by many local ...,573440114776f41900661b03,What is Tucson's Fox station?,"Tucson,_Arizona"
4,[367],[305],Situated on one of the world's largest natural...,56ce3124aab44d1400b8852d,What is the size of New York City in square mi...,New_York_City
...,...,...,...,...,...,...
495,[168],[individual worth],"In a 1964 interview, Lee remarked that her asp...",56d19e43e7d4791d0090207c,Both authors valued what over social standing?,To_Kill_a_Mockingbird
496,[326],[A planned 11.7 km (7.3 mi) branch to Midford ...,The Somerset Coal Canal was built in the early...,5726b4e8dd62a815002e8d86,What was never built,Somerset
497,[253],[eastern],"On November 21, 1789, North Carolina became th...",57278f03f1498d1400e8fc16,What part of the state were most slave and pla...,North_Carolina
498,[122],[Secretary of State for Education],The largest expansion of comprehensive schools...,56dc7c3114d3a41400c268f0,what was Anthony Crosland's role in government?,Comprehensive_school


In [32]:
df['request'] = df.context + '\n\nQuestion: ' + df.question

In [33]:
df['response'] = df.ans_texts.str[0]

In [34]:
df = df.sample(df.shape[0])

In [35]:
data = [{'request': t.request, 'response': t.response} for t in df[['request', 'response']].itertuples()]

In [36]:
train = data[:400]

In [39]:
train = json.load(open('train.json', 'r'))

In [42]:
sample = df[~df.request.isin([t['request'] for t in train])].sample().iloc[0]
print(sample['request'])
print('-----')
print(sample['response'])

Early releases of the Washington coin included error coins shipped primarily from the Philadelphia mint to Florida and Tennessee banks. Highly sought after by collectors, and trading for as much as $850 each within a week of discovery, the error coins were identified by the absence of the edge impressions "E PLURIBUS UNUM IN GOD WE TRUST 2007 P". The mint of origin is generally accepted to be mostly Philadelphia, although identifying the source mint is impossible without opening a mint pack also containing marked units. Edge lettering is minted in both orientations with respect to "heads", some amateur collectors were initially duped into buying "upside down lettering error" coins. Some cynics also erroneously point out that the Federal Reserve makes more profit from dollar bills than dollar coins because they wear out in a few years, whereas coins are more permanent. The fallacy of this argument arises because new notes printed to replace worn out notes, which have been withdrawn from

In [ ]:
# Napoleon won victories at Austerlitz in 1805 and Jena in 1806.

In [46]:
result = requests.post(
    url='https://llm.api.cloud.yandex.net/llm/v1alpha/instruct',
    headers={'Authorization': f'Bearer {iamtoken}', 'x-folder-id': dir_id},
    json={
      "model": "general",
      "instruction_text": sample['request'],
      #"request_text": f"{sample['request']}",
      "generation_options": {
        "max_tokens": 1500,  
        "temperature": 0.5
      }
    }
)

In [47]:
print(result.json()['result']['alternatives'][0]['text'])

Vending machines commonly only accept $1 bills.
